In [1]:
#######################################################################
###############           BRDF CORRECTION         #####################
#######################################################################
## Daniel Borini Alves; Dhemerson Conciani; and Swanni Alvarado

## --  adapted  based   on  Minh Nguyen  script   <https://github.com/ndminhhus/geeguide/blob/master/04.topo_correction.md>  and 
## Poortinga et al. (2018). Original background developed by Roy et al (2017) for Landsat data, and also successfully applied over 
## Sentinel 2 datasets (Zhang et al., 2018). Compute and correct Bidirectional Reflectance Distribution Function (BRDF) effects.
## See all references in the end of the script.

## Inputs: Bottom of Atmosphere (BOA) reflectance (PY6s), adjusted geometric (AROSICS), of each data collection (Landsat and Sentinel)
## Output: BOA reflectance, ajusted geometric and BRDF 

## ACTIVATE PACKAGES
import datetime
import math
import os
import sys
import ee
import geebam
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as ro

ee.Initialize()

In [2]:
base = importr ('base')
filesstrings = importr ('filesstrings')

In [95]:
## Define spatio-temporal filters (ensure that are the same as BOA correction)
startDate = ee.Date('2019-08-01')
endDate   = ee.Date('2019-10-01') 
geom = ee.Geometry.Point(-61.5775286643,-8.4852883834)
region = ee.FeatureCollection("users/shps/Recorte_ZonasAeB_Sent2")

In [96]:
## Sentinel 2B collection images
sentinel_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom).filterDate(startDate,endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',100))

## Landsat OLI collection images
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)

## Landsat ETM+ collection images
landsat7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)

In [97]:
## Extract Sentinel metadata
GRANULE_ID = sentinel_collection.aggregate_array('GRANULE_ID').getInfo()
CLOUD_COVERAGE_ASSESSMENT = sentinel_collection.aggregate_array('CLOUD_COVERAGE_ASSESSMENT').getInfo()
SPACECRAFT_NAME = sentinel_collection.aggregate_array('SPACECRAFT_NAME').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A').getInfo()
MEAN_SOLAR_AZIMUTH_ANGLE = sentinel_collection.aggregate_array('MEAN_SOLAR_AZIMUTH_ANGLE').getInfo()
SOLAR_IRRADIANCE_B12 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B12').getInfo()
SOLAR_IRRADIANCE_B10 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B10').getInfo()
SENSOR_QUALITY = sentinel_collection.aggregate_array('SENSOR_QUALITY').getInfo()
SOLAR_IRRADIANCE_B11 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B11').getInfo()
GENERATION_TIME = sentinel_collection.aggregate_array('GENERATION_TIME').getInfo()
SOLAR_IRRADIANCE_B8A = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B8A').getInfo()
FORMAT_CORRECTNESS = sentinel_collection.aggregate_array('FORMAT_CORRECTNESS').getInfo()
CLOUD_COVERAGE_ASSESSMENT = sentinel_collection.aggregate_array('CLOUD_COVERAGE_ASSESSMENT').getInfo()
DATASTRIP_ID = sentinel_collection.aggregate_array('DATASTRIP_ID').getInfo()
PROCESSING_BASELINE = sentinel_collection.aggregate_array('PROCESSING_BASELINE').getInfo()
SENSING_ORBIT_NUMBER = sentinel_collection.aggregate_array('SENSING_ORBIT_NUMBER').getInfo()
SENSING_ORBIT_DIRECTION = sentinel_collection.aggregate_array('SENSING_ORBIT_DIRECTION').getInfo()
GENERAL_QUALITY = sentinel_collection.aggregate_array('GENERAL_QUALITY').getInfo()
REFLECTANCE_CONVERSION_CORRECTION = sentinel_collection.aggregate_array('REFLECTANCE_CONVERSION_CORRECTION').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B8 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8').getInfo()
DATATAKE_TYPE = sentinel_collection.aggregate_array('DATATAKE_TYPE').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B9 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B9').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B6 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B6').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B7 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B7').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B4 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B4').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B1 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B1').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B5 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5').getInfo()
RADIOMETRIC_QUALITY = sentinel_collection.aggregate_array('RADIOMETRIC_QUALITY').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B2 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B2').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B3 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B3').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B5 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B5').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B4 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B4').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B3 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B3').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B2 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B2').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B9 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B9').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B8 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B8').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B7 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B7').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B6 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B6').getInfo()
MEAN_SOLAR_ZENITH_ANGLE = sentinel_collection.aggregate_array('MEAN_SOLAR_ZENITH_ANGLE').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B8A = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B8A').getInfo()
MGRS_TILE = sentinel_collection.aggregate_array('MGRS_TILE').getInfo()
CLOUDY_PIXEL_PERCENTAGE = sentinel_collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()
PRODUCT_ID = sentinel_collection.aggregate_array('PRODUCT_ID').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B10 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B10').getInfo()
SOLAR_IRRADIANCE_B9 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B9').getInfo()
DEGRADED_MSI_DATA_PERCENTAGE = sentinel_collection.aggregate_array('DEGRADED_MSI_DATA_PERCENTAGE').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B11 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B11').getInfo()
MEAN_INCIDENCE_ZENITH_ANGLE_B12 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_ZENITH_ANGLE_B12').getInfo()
SOLAR_IRRADIANCE_B6 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B6').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B10 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B10').getInfo()
SOLAR_IRRADIANCE_B5 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B5').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B11 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B11').getInfo()
SOLAR_IRRADIANCE_B8 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B8').getInfo()
MEAN_INCIDENCE_AZIMUTH_ANGLE_B12 = sentinel_collection.aggregate_array('MEAN_INCIDENCE_AZIMUTH_ANGLE_B12').getInfo()
SOLAR_IRRADIANCE_B7 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B7').getInfo()
SOLAR_IRRADIANCE_B2 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B2').getInfo()
SOLAR_IRRADIANCE_B1 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B1').getInfo()
SOLAR_IRRADIANCE_B4 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B4').getInfo()
GEOMETRIC_QUALITY = sentinel_collection.aggregate_array('GEOMETRIC_QUALITY').getInfo()
SOLAR_IRRADIANCE_B3 = sentinel_collection.aggregate_array('SOLAR_IRRADIANCE_B3').getInfo()
asset_size = sentinel_collection.aggregate_array('system:asset_size').getInfo()
footprint = sentinel_collection.aggregate_array('system:footprint').getInfo()
index = sentinel_collection.aggregate_array('system:index').getInfo()
time_end = sentinel_collection.aggregate_array('system:time_end').getInfo()
time_start = sentinel_collection.aggregate_array('system:time_start').getInfo()

## Insert suffix 
GRANULE_ID = [s + '_BOA_GEOM' for s in GRANULE_ID]

## Export GEE Sentinel metadata as CSV 
METADATA_ARRAY = {'id': GRANULE_ID, 'CLOUD_COVERAGE_ASSESSMENT': CLOUD_COVERAGE_ASSESSMENT, 'SPACECRAFT_NAME' : SPACECRAFT_NAME, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A, 'MEAN_SOLAR_AZIMUTH_ANGLE' : MEAN_SOLAR_AZIMUTH_ANGLE, 'SOLAR_IRRADIANCE_B12' : SOLAR_IRRADIANCE_B12, 'SOLAR_IRRADIANCE_B10' : SOLAR_IRRADIANCE_B10, 'SENSOR_QUALITY' :SENSOR_QUALITY, 'SOLAR_IRRADIANCE_B11' : SOLAR_IRRADIANCE_B11, 'GENERATION_TIME' : GENERATION_TIME, 'SOLAR_IRRADIANCE_B8A' : SOLAR_IRRADIANCE_B8A, 'FORMAT_CORRECTNESS' : FORMAT_CORRECTNESS, 'CLOUD_COVERAGE_ASSESSMENT' : CLOUD_COVERAGE_ASSESSMENT, 'DATASTRIP_ID' :DATASTRIP_ID, 'PROCESSING_BASELINE' :  PROCESSING_BASELINE,'SENSING_ORBIT_NUMBER' : SENSING_ORBIT_NUMBER, 'SENSING_ORBIT_DIRECTION' : SENSING_ORBIT_DIRECTION, 'GENERAL_QUALITY' : GENERAL_QUALITY, 'REFLECTANCE_CONVERSION_CORRECTION' : REFLECTANCE_CONVERSION_CORRECTION, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B8 , 'DATATAKE_TYPE' : DATATAKE_TYPE, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B9, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B6, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B7, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B4, 'MEAN_INCIDENCE_ZENITH_ANGLE_B1' : MEAN_INCIDENCE_ZENITH_ANGLE_B1, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B5, 'RADIOMETRIC_QUALITY' : RADIOMETRIC_QUALITY, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B2, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B3, 'MEAN_INCIDENCE_ZENITH_ANGLE_B5' : MEAN_INCIDENCE_ZENITH_ANGLE_B5, 'MEAN_INCIDENCE_ZENITH_ANGLE_B4' : MEAN_INCIDENCE_ZENITH_ANGLE_B4 , 'MEAN_INCIDENCE_ZENITH_ANGLE_B3' : MEAN_INCIDENCE_ZENITH_ANGLE_B3, 'MEAN_INCIDENCE_ZENITH_ANGLE_B2' : MEAN_INCIDENCE_ZENITH_ANGLE_B2, 'MEAN_INCIDENCE_ZENITH_ANGLE_B9' : MEAN_INCIDENCE_ZENITH_ANGLE_B9, 'MEAN_INCIDENCE_ZENITH_ANGLE_B8' : MEAN_INCIDENCE_ZENITH_ANGLE_B8, 'MEAN_INCIDENCE_ZENITH_ANGLE_B7' : MEAN_INCIDENCE_ZENITH_ANGLE_B7, 'MEAN_INCIDENCE_ZENITH_ANGLE_B6' : MEAN_INCIDENCE_ZENITH_ANGLE_B6, 'MEAN_SOLAR_ZENITH_ANGLE' : MEAN_SOLAR_ZENITH_ANGLE, 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A' : MEAN_INCIDENCE_ZENITH_ANGLE_B8A, 'MGRS_TILE' : MGRS_TILE, 'CLOUDY_PIXEL_PERCENTAGE' : CLOUDY_PIXEL_PERCENTAGE, 'PRODUCT_ID' : PRODUCT_ID, 'MEAN_INCIDENCE_ZENITH_ANGLE_B10' : MEAN_INCIDENCE_ZENITH_ANGLE_B10, 'SOLAR_IRRADIANCE_B9' : SOLAR_IRRADIANCE_B9, 'DEGRADED_MSI_DATA_PERCENTAGE': DEGRADED_MSI_DATA_PERCENTAGE, 'MEAN_INCIDENCE_ZENITH_ANGLE_B11' : MEAN_INCIDENCE_ZENITH_ANGLE_B11, 'MEAN_INCIDENCE_ZENITH_ANGLE_B12' : MEAN_INCIDENCE_ZENITH_ANGLE_B12, 'SOLAR_IRRADIANCE_B6' : SOLAR_IRRADIANCE_B6, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B10, 'SOLAR_IRRADIANCE_B5' : SOLAR_IRRADIANCE_B5, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B11, 'SOLAR_IRRADIANCE_B8' : SOLAR_IRRADIANCE_B8, 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12' : MEAN_INCIDENCE_AZIMUTH_ANGLE_B12,  'SOLAR_IRRADIANCE_B7' : SOLAR_IRRADIANCE_B7, 'SOLAR_IRRADIANCE_B2' : SOLAR_IRRADIANCE_B2, 'SOLAR_IRRADIANCE_B1' : SOLAR_IRRADIANCE_B1, 'SOLAR_IRRADIANCE_B4' : SOLAR_IRRADIANCE_B4, 'GEOMETRIC_QUALITY' : GEOMETRIC_QUALITY, 'SOLAR_IRRADIANCE_B3':SOLAR_IRRADIANCE_B3, 'asset_size':asset_size, 'index':index , 'time_end': time_end, 'time_start':time_start }
METADATA_DF = pd.DataFrame(data=METADATA_ARRAY)
METADATA_DF.to_csv(path_or_buf= 'C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/sentinel_meta.csv', index= False)
METADATA_DF

,id,CLOUD_COVERAGE_ASSESSMENT,SPACECRAFT_NAME,MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A,MEAN_SOLAR_AZIMUTH_ANGLE,SOLAR_IRRADIANCE_B12,SOLAR_IRRADIANCE_B10,SENSOR_QUALITY,SOLAR_IRRADIANCE_B11,GENERATION_TIME,...,SOLAR_IRRADIANCE_B7,SOLAR_IRRADIANCE_B2,SOLAR_IRRADIANCE_B1,SOLAR_IRRADIANCE_B4,GEOMETRIC_QUALITY,SOLAR_IRRADIANCE_B3,asset_size,index,time_end,time_start
0,L1C_T20LPR_A021500_20190804T143101_BOA_GEOM,14.1260,Sentinel-2A,284.240750,42.919489,85.25,367.15,PASSED,245.59,1564941038000,...,1162.08,1959.72,1884.69,1512.06,PASSED,1823.24,1048275501,20190804T142801_20190804T143101_T20LPR,1564929309667,1564929309667
1,L1C_T20LPR_A012663_20190809T142759_BOA_GEOM,0.0797,Sentinel-2B,283.215434,44.468791,87.75,365.41,PASSED,247.08,1565366995000,...,1175.57,1959.77,1874.30,1512.79,PASSED,1824.93,1034207090,20190809T142759_20190809T142759_T20LPR,1565361312373,1565361312373
2,L1C_T20LPR_A021643_20190814T142756_BOA_GEOM,0.0000,Sentinel-2A,284.188779,46.202360,85.25,367.15,PASSED,245.59,1565799010000,...,1162.08,1959.72,1884.69,1512.06,PASSED,1823.24,1035308348,20190814T142801_20190814T142756_T20LPR,1565793308622,1565793308622
3,L1C_T20LPR_A012806_20190819T142758_BOA_GEOM,10.8448,Sentinel-2B,283.280238,48.078412,87.75,365.41,PASSED,247.08,1566236992000,...,1175.57,1959.77,1874.30,1512.79,PASSED,1824.93,1070705358,20190819T142759_20190819T142758_T20LPR,1566225311012,1566225311012
4,L1C_T20LPR_A021786_20190824T142754_BOA_GEOM,2.1027,Sentinel-2A,284.071729,50.185067,85.25,367.15,PASSED,245.59,1566662954000,...,1162.08,1959.72,1884.69,1512.06,PASSED,1823.24,974062658,20190824T142751_20190824T142754_T20LPR,1566657307160,1566657307160
5,L1C_T20LPR_A012949_20190829T142756_BOA_GEOM,0.0608,Sentinel-2B,283.224706,52.491767,87.75,365.41,PASSED,247.08,1567094911000,...,1175.57,1959.77,1874.30,1512.79,PASSED,1824.93,1025018142,20190829T142759_20190829T142756_T20LPR,1567089309221,1567089309221
6,L1C_T20LPR_A021929_20190903T142959_BOA_GEOM,11.6291,Sentinel-2A,284.186049,55.091110,85.25,367.15,PASSED,245.59,1567533199000,...,1162.08,1959.72,1884.69,1512.06,PASSED,1823.24,1066104326,20190903T142751_20190903T142959_T20LPR,1567521305312,1567521305312
7,L1C_T20LPR_A013092_20190908T142754_BOA_GEOM,0.1021,Sentinel-2B,283.290484,57.960798,87.75,365.41,PASSED,247.08,1567959078000,...,1175.57,1959.77,1874.30,1512.79,PASSED,1824.93,1064650947,20190908T142759_20190908T142754_T20LPR,1567953306979,1567953306979
8,L1C_T20LPR_A022072_20190913T142751_BOA_GEOM,0.0698,Sentinel-2A,284.190147,61.194149,85.25,367.15,PASSED,245.59,1568390858000,...,1162.08,1959.72,1884.69,1512.06,PASSED,1823.24,1013964034,20190913T142751_20190913T142751_T20LPR,1568385303911,1568385303911
9,L1C_T20LPR_A013235_20190918T143240_BOA_GEOM,96.3869,Sentinel-2B,283.290374,64.794385,87.75,365.41,PASSED,247.08,1568834808000,...,1175.57,1959.77,1874.30,1512.79,PASSED,1824.93,1059553337,20190918T142749_20190918T143240_T20LPR,1568817304315,1568817304315


In [98]:
## Extract Landsat ETM+ metadata
LANDSAT_PRODUCT_ID = landsat7_collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()
CLOUD_COVER = landsat7_collection.aggregate_array('CLOUD_COVER').getInfo()
CLOUD_COVER_LAND = landsat7_collection.aggregate_array('CLOUD_COVER_LAND').getInfo()
COLLECTION_CATEGORY = landsat7_collection.aggregate_array('COLLECTION_CATEGORY').getInfo()
COLLECTION_NUMBER = landsat7_collection.aggregate_array('COLLECTION_NUMBER').getInfo()
CORRECTION_BIAS_BAND_1 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_1').getInfo()
CORRECTION_BIAS_BAND_2 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_2').getInfo()
CORRECTION_BIAS_BAND_3 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_3').getInfo()
CORRECTION_BIAS_BAND_4 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_4').getInfo()
CORRECTION_BIAS_BAND_5 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_5').getInfo()
CORRECTION_BIAS_BAND_6_VCID_1 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_6_VCID_1').getInfo()
CORRECTION_BIAS_BAND_6_VCID_2 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_6_VCID_2').getInfo()
CORRECTION_BIAS_BAND_7 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_7').getInfo()
CORRECTION_BIAS_BAND_8 = landsat7_collection.aggregate_array('CORRECTION_BIAS_BAND_8').getInfo()
CORRECTION_GAIN_BAND_1 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_1').getInfo()
CORRECTION_GAIN_BAND_2 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_2').getInfo()
CORRECTION_GAIN_BAND_3 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_3').getInfo()
CORRECTION_GAIN_BAND_4 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_4').getInfo()
CORRECTION_GAIN_BAND_5 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_5').getInfo()
CORRECTION_GAIN_BAND_6_VCID_1 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_6_VCID_1').getInfo()
CORRECTION_GAIN_BAND_6_VCID_2 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_6_VCID_2').getInfo()
CORRECTION_GAIN_BAND_7 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_7').getInfo()
CORRECTION_GAIN_BAND_8 = landsat7_collection.aggregate_array('CORRECTION_GAIN_BAND_8').getInfo()
CPF_NAME = landsat7_collection.aggregate_array('CPF_NAME').getInfo()
DATA_CATEGORY = landsat7_collection.aggregate_array('DATA_CATEGORY').getInfo()
DATA_TYPE = landsat7_collection.aggregate_array('DATA_TYPE').getInfo()
DATE_ACQUIRED = landsat7_collection.aggregate_array('DATE_ACQUIRED').getInfo()
DATUM = landsat7_collection.aggregate_array('DATUM').getInfo()
EARTH_SUN_DISTANCE = landsat7_collection.aggregate_array('EARTH_SUN_DISTANCE').getInfo()
ELEVATION_SOURCE = landsat7_collection.aggregate_array('ELEVATION_SOURCE').getInfo()
ELLIPSOID = landsat7_collection.aggregate_array('ELLIPSOID').getInfo()
EPHEMERIS_TYPE = landsat7_collection.aggregate_array('EPHEMERIS_TYPE').getInfo()
FILE_DATE = landsat7_collection.aggregate_array('FILE_DATE').getInfo()
GAIN_BAND_1 = landsat7_collection.aggregate_array('GAIN_BAND_1').getInfo()
GAIN_BAND_2 = landsat7_collection.aggregate_array('GAIN_BAND_2').getInfo()
GAIN_BAND_3 = landsat7_collection.aggregate_array('GAIN_BAND_3').getInfo()
GAIN_BAND_4 = landsat7_collection.aggregate_array('GAIN_BAND_4').getInfo()
GAIN_BAND_5 = landsat7_collection.aggregate_array('GAIN_BAND_5').getInfo()
GAIN_BAND_6_VCID_1 = landsat7_collection.aggregate_array('GAIN_BAND_6_VCID_1').getInfo()
GAIN_BAND_6_VCID_2 = landsat7_collection.aggregate_array('GAIN_BAND_6_VCID_2').getInfo()
GAIN_BAND_7 = landsat7_collection.aggregate_array('GAIN_BAND_7').getInfo()
GAIN_BAND_8 = landsat7_collection.aggregate_array('GAIN_BAND_8').getInfo()
GAIN_CHANGE_BAND_1 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_1').getInfo()
GAIN_CHANGE_BAND_2 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_2').getInfo()
GAIN_CHANGE_BAND_3 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_3').getInfo()
GAIN_CHANGE_BAND_4 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_4').getInfo()
GAIN_CHANGE_BAND_5 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_5').getInfo()
GAIN_CHANGE_BAND_6_VCID_1 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_6_VCID_1').getInfo()
GAIN_CHANGE_BAND_6_VCID_2 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_6_VCID_2').getInfo()
GAIN_CHANGE_BAND_7 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_7').getInfo()
GAIN_CHANGE_BAND_8 = landsat7_collection.aggregate_array('GAIN_CHANGE_BAND_8').getInfo()
GAIN_CHANGE_SCAN_BAND_1 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_1').getInfo()
GAIN_CHANGE_SCAN_BAND_2 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_2').getInfo()
GAIN_CHANGE_SCAN_BAND_3 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_3').getInfo()
GAIN_CHANGE_SCAN_BAND_4 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_4').getInfo()
GAIN_CHANGE_SCAN_BAND_5 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_5').getInfo()
GAIN_CHANGE_SCAN_BAND_6_VCID_1 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_6_VCID_1').getInfo()
GAIN_CHANGE_SCAN_BAND_6_VCID_2 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_6_VCID_2').getInfo()
GAIN_CHANGE_SCAN_BAND_7 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_7').getInfo()
GAIN_CHANGE_SCAN_BAND_8 = landsat7_collection.aggregate_array('GAIN_CHANGE_SCAN_BAND_8').getInfo()
GEOMETRIC_RMSE_MODEL = landsat7_collection.aggregate_array('GEOMETRIC_RMSE_MODEL').getInfo()
GEOMETRIC_RMSE_MODEL_X = landsat7_collection.aggregate_array('GEOMETRIC_RMSE_MODEL_X').getInfo()
GEOMETRIC_RMSE_MODEL_Y = landsat7_collection.aggregate_array('GEOMETRIC_RMSE_MODEL_Y').getInfo()
GRID_CELL_SIZE_PANCHROMATIC = landsat7_collection.aggregate_array('GRID_CELL_SIZE_PANCHROMATIC').getInfo()
GRID_CELL_SIZE_REFLECTIVE = landsat7_collection.aggregate_array('GRID_CELL_SIZE_REFLECTIVE').getInfo()
GRID_CELL_SIZE_THERMAL = landsat7_collection.aggregate_array('GRID_CELL_SIZE_THERMAL').getInfo()
GROUND_CONTROL_POINTS_MODEL = landsat7_collection.aggregate_array('GROUND_CONTROL_POINTS_MODEL').getInfo()
GROUND_CONTROL_POINTS_VERIFY = landsat7_collection.aggregate_array('GROUND_CONTROL_POINTS_VERIFY').getInfo()
GROUND_CONTROL_POINTS_VERSION = landsat7_collection.aggregate_array('GROUND_CONTROL_POINTS_VERSION').getInfo()
IMAGE_QUALITY = landsat7_collection.aggregate_array('IMAGE_QUALITY').getInfo()
K1_CONSTANT_BAND_6_VCID_1 = landsat7_collection.aggregate_array('K1_CONSTANT_BAND_6_VCID_1').getInfo()
K1_CONSTANT_BAND_6_VCID_2 = landsat7_collection.aggregate_array('K1_CONSTANT_BAND_6_VCID_2').getInfo()
K2_CONSTANT_BAND_6_VCID_1 = landsat7_collection.aggregate_array('K2_CONSTANT_BAND_6_VCID_1').getInfo()
K2_CONSTANT_BAND_6_VCID_2 = landsat7_collection.aggregate_array('K2_CONSTANT_BAND_6_VCID_2').getInfo()
LANDSAT_SCENE_ID = landsat7_collection.aggregate_array('LANDSAT_SCENE_ID').getInfo()
MAP_PROJECTION = landsat7_collection.aggregate_array('MAP_PROJECTION').getInfo()
MAP_PROJECTION_L0RA = landsat7_collection.aggregate_array('MAP_PROJECTION_L0RA').getInfo()
ORIENTATION = landsat7_collection.aggregate_array('ORIENTATION').getInfo()
PANCHROMATIC_LINES = landsat7_collection.aggregate_array('PANCHROMATIC_LINES').getInfo()
PANCHROMATIC_SAMPLES = landsat7_collection.aggregate_array('PANCHROMATIC_SAMPLES').getInfo()
PROCESSING_SOFTWARE_VERSION = landsat7_collection.aggregate_array('PROCESSING_SOFTWARE_VERSION').getInfo()
RADIANCE_ADD_BAND_1 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_1').getInfo()
RADIANCE_ADD_BAND_2 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_2').getInfo()
RADIANCE_ADD_BAND_3 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_3').getInfo()
RADIANCE_ADD_BAND_4 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_4').getInfo()
RADIANCE_ADD_BAND_5 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_5').getInfo()
RADIANCE_ADD_BAND_6_VCID_1 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_6_VCID_1').getInfo()
RADIANCE_ADD_BAND_6_VCID_2 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_6_VCID_2').getInfo()
RADIANCE_ADD_BAND_7 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_7').getInfo()
RADIANCE_ADD_BAND_8 = landsat7_collection.aggregate_array('RADIANCE_ADD_BAND_8').getInfo()
RADIANCE_MAXIMUM_BAND_1 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_1').getInfo()
RADIANCE_MAXIMUM_BAND_2 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_2').getInfo()
RADIANCE_MAXIMUM_BAND_3 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_3').getInfo()
RADIANCE_MAXIMUM_BAND_4 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_4').getInfo()
RADIANCE_MAXIMUM_BAND_5 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_5').getInfo()
RADIANCE_MAXIMUM_BAND_6_VCID_1 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_6_VCID_1').getInfo()
RADIANCE_MAXIMUM_BAND_6_VCID_2 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_6_VCID_2').getInfo()
RADIANCE_MAXIMUM_BAND_7 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_7').getInfo()
RADIANCE_MAXIMUM_BAND_8 = landsat7_collection.aggregate_array('RADIANCE_MAXIMUM_BAND_8').getInfo()
RADIANCE_MINIMUM_BAND_1 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_1').getInfo()
RADIANCE_MINIMUM_BAND_2 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_2').getInfo()
RADIANCE_MINIMUM_BAND_3 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_3').getInfo()
RADIANCE_MINIMUM_BAND_4 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_4').getInfo()
RADIANCE_MINIMUM_BAND_5 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_5').getInfo()
RADIANCE_MINIMUM_BAND_6_VCID_1 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_6_VCID_1').getInfo()
RADIANCE_MINIMUM_BAND_6_VCID_2 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_6_VCID_2').getInfo()
RADIANCE_MINIMUM_BAND_7 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_7').getInfo()
RADIANCE_MINIMUM_BAND_8 = landsat7_collection.aggregate_array('RADIANCE_MINIMUM_BAND_8').getInfo()
RADIANCE_MULT_BAND_1 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_1').getInfo()
RADIANCE_MULT_BAND_2 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_2').getInfo()
RADIANCE_MULT_BAND_3 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_3').getInfo()
RADIANCE_MULT_BAND_4 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_4').getInfo()
RADIANCE_MULT_BAND_5 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_5').getInfo()
RADIANCE_MULT_BAND_6_VCID_1 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_6_VCID_1').getInfo()
RADIANCE_MULT_BAND_6_VCID_2 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_6_VCID_2').getInfo()
RADIANCE_MULT_BAND_7 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_7').getInfo()
RADIANCE_MULT_BAND_8 = landsat7_collection.aggregate_array('RADIANCE_MULT_BAND_8').getInfo()
REFLECTANCE_ADD_BAND_1 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_1').getInfo()
REFLECTANCE_ADD_BAND_2 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_2').getInfo()
REFLECTANCE_ADD_BAND_3 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_3').getInfo()
REFLECTANCE_ADD_BAND_4 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_4').getInfo()
REFLECTANCE_ADD_BAND_5 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_5').getInfo()
REFLECTANCE_ADD_BAND_7 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_7').getInfo()
REFLECTANCE_ADD_BAND_8 = landsat7_collection.aggregate_array('REFLECTANCE_ADD_BAND_8').getInfo()
REFLECTANCE_MAXIMUM_BAND_1 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_1').getInfo()
REFLECTANCE_MAXIMUM_BAND_2 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_2').getInfo()
REFLECTANCE_MAXIMUM_BAND_3 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_3').getInfo()
REFLECTANCE_MAXIMUM_BAND_4 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_4').getInfo()
REFLECTANCE_MAXIMUM_BAND_5 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_5').getInfo()
REFLECTANCE_MAXIMUM_BAND_7 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_7').getInfo()
REFLECTANCE_MAXIMUM_BAND_8 = landsat7_collection.aggregate_array('REFLECTANCE_MAXIMUM_BAND_8').getInfo()
REFLECTANCE_MINIMUM_BAND_1 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_1').getInfo()
REFLECTANCE_MINIMUM_BAND_2 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_2').getInfo()
REFLECTANCE_MINIMUM_BAND_3 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_3').getInfo()
REFLECTANCE_MINIMUM_BAND_4 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_4').getInfo()
REFLECTANCE_MINIMUM_BAND_5 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_5').getInfo()
REFLECTANCE_MINIMUM_BAND_7 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_7').getInfo()
REFLECTANCE_MINIMUM_BAND_8 = landsat7_collection.aggregate_array('REFLECTANCE_MINIMUM_BAND_8').getInfo()
REFLECTANCE_MULT_BAND_1 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_1').getInfo()
REFLECTANCE_MULT_BAND_2 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_2').getInfo()
REFLECTANCE_MULT_BAND_3 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_3').getInfo()
REFLECTANCE_MULT_BAND_4 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_4').getInfo()
REFLECTANCE_MULT_BAND_5 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_5').getInfo()
REFLECTANCE_MULT_BAND_7 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_7').getInfo()
REFLECTANCE_MULT_BAND_8 = landsat7_collection.aggregate_array('REFLECTANCE_MULT_BAND_8').getInfo()
REFLECTIVE_LINES = landsat7_collection.aggregate_array('REFLECTIVE_LINES').getInfo()
REFLECTIVE_SAMPLES = landsat7_collection.aggregate_array('REFLECTIVE_SAMPLES').getInfo()
REQUEST_ID = landsat7_collection.aggregate_array('REQUEST_ID').getInfo()
SATURATION_BAND_1 = landsat7_collection.aggregate_array('SATURATION_BAND_1').getInfo()
SATURATION_BAND_2 = landsat7_collection.aggregate_array('SATURATION_BAND_2').getInfo()
SATURATION_BAND_3 = landsat7_collection.aggregate_array('SATURATION_BAND_3').getInfo()
SATURATION_BAND_4 = landsat7_collection.aggregate_array('SATURATION_BAND_4').getInfo()
SATURATION_BAND_5 = landsat7_collection.aggregate_array('SATURATION_BAND_5').getInfo()
SATURATION_BAND_6_VCID_1 = landsat7_collection.aggregate_array('SATURATION_BAND_6_VCID_1').getInfo()
SATURATION_BAND_6_VCID_2 = landsat7_collection.aggregate_array('SATURATION_BAND_6_VCID_2').getInfo()
SATURATION_BAND_7 = landsat7_collection.aggregate_array('SATURATION_BAND_7').getInfo()
SATURATION_BAND_8 = landsat7_collection.aggregate_array('SATURATION_BAND_8').getInfo()
SCENE_CENTER_TIME = landsat7_collection.aggregate_array('SCENE_CENTER_TIME').getInfo()
SENSOR_ID = landsat7_collection.aggregate_array('SENSOR_ID').getInfo()
SENSOR_MODE = landsat7_collection.aggregate_array('SENSOR_MODE').getInfo()
SPACECRAFT_ID = landsat7_collection.aggregate_array('SPACECRAFT_ID').getInfo()
STATION_ID = landsat7_collection.aggregate_array('STATION_ID').getInfo()
SUN_AZIMUTH = landsat7_collection.aggregate_array('SUN_AZIMUTH').getInfo()
SUN_ELEVATION = landsat7_collection.aggregate_array('SUN_ELEVATION').getInfo()
THERMAL_LINES = landsat7_collection.aggregate_array('THERMAL_LINES').getInfo()
THERMAL_SAMPLES = landsat7_collection.aggregate_array('THERMAL_SAMPLES').getInfo()
UTM_ZONE = landsat7_collection.aggregate_array('UTM_ZONE').getInfo()
WRS_PATH = landsat7_collection.aggregate_array('WRS_PATH').getInfo()
WRS_ROW = landsat7_collection.aggregate_array('WRS_ROW').getInfo()
index = landsat7_collection.aggregate_array('system:index').getInfo() 
time_end = landsat7_collection.aggregate_array('system:time_end').getInfo() 
time_start = landsat7_collection.aggregate_array('system:time_start').getInfo() 

## Insert suffix 
LANDSAT_PRODUCT_ID = [s + '_BOA_GEOM' for s in LANDSAT_PRODUCT_ID]

## Export GEE Sentinel metadata as CSV 
METADATA_ARRAY = { 'id' : LANDSAT_PRODUCT_ID, 'CLOUD_COVER':CLOUD_COVER, 'CLOUD_COVER_LAND' : CLOUD_COVER_LAND, 'COLLECTION_CATEGORY': COLLECTION_CATEGORY, 'COLLECTION_NUMBER': COLLECTION_NUMBER, 'CORRECTION_BIAS_BAND_1': CORRECTION_BIAS_BAND_1, 'CORRECTION_BIAS_BAND_2':CORRECTION_BIAS_BAND_2, 'CORRECTION_BIAS_BAND_3': CORRECTION_BIAS_BAND_3, 'CORRECTION_BIAS_BAND_4': CORRECTION_BIAS_BAND_4, 'CORRECTION_BIAS_BAND_5':CORRECTION_BIAS_BAND_5, 'CORRECTION_BIAS_BAND_6_VCID_1':CORRECTION_BIAS_BAND_6_VCID_1, 'CORRECTION_BIAS_BAND_6_VCID_2': CORRECTION_BIAS_BAND_6_VCID_2, 'CORRECTION_BIAS_BAND_7':CORRECTION_BIAS_BAND_7, 'CORRECTION_BIAS_BAND_8':CORRECTION_BIAS_BAND_8, 'CORRECTION_GAIN_BAND_1':CORRECTION_GAIN_BAND_1, 'CORRECTION_GAIN_BAND_2':CORRECTION_GAIN_BAND_2, 'CORRECTION_GAIN_BAND_3':CORRECTION_GAIN_BAND_3, 'CORRECTION_GAIN_BAND_4':CORRECTION_GAIN_BAND_4, 'CORRECTION_GAIN_BAND_5':CORRECTION_GAIN_BAND_5, 'CORRECTION_GAIN_BAND_6_VCID_1':CORRECTION_GAIN_BAND_6_VCID_1, 'CORRECTION_GAIN_BAND_6_VCID_2':CORRECTION_GAIN_BAND_6_VCID_2, 'CORRECTION_GAIN_BAND_7':CORRECTION_GAIN_BAND_7, 'CORRECTION_GAIN_BAND_8':CORRECTION_GAIN_BAND_8, 'CPF_NAME':CPF_NAME, 'DATA_CATEGORY':DATA_CATEGORY, 'DATA_TYPE':DATA_TYPE, 'DATE_ACQUIRED':DATE_ACQUIRED, 'DATUM':DATUM, 'EARTH_SUN_DISTANCE':EARTH_SUN_DISTANCE, 'ELEVATION_SOURCE':ELEVATION_SOURCE, 'ELLIPSOID':ELLIPSOID, 'EPHEMERIS_TYPE':EPHEMERIS_TYPE, 'FILE_DATE':FILE_DATE, 'GAIN_BAND_1':GAIN_BAND_1, 'GAIN_BAND_2':GAIN_BAND_2, 'GAIN_BAND_3':GAIN_BAND_3, 'GAIN_BAND_4':GAIN_BAND_4, 'GAIN_BAND_5':GAIN_BAND_5, 'GAIN_BAND_6_VCID_1':GAIN_BAND_6_VCID_1, 'GAIN_BAND_6_VCID_2':GAIN_BAND_6_VCID_2, 'GAIN_BAND_7':GAIN_BAND_7, 'GAIN_BAND_8':GAIN_BAND_8, 'GAIN_CHANGE_BAND_1':GAIN_CHANGE_BAND_1, 'GAIN_CHANGE_BAND_2':GAIN_CHANGE_BAND_2, 'GAIN_CHANGE_BAND_3':GAIN_CHANGE_BAND_3, 'GAIN_CHANGE_BAND_4':GAIN_CHANGE_BAND_4, 'GAIN_CHANGE_BAND_5':GAIN_CHANGE_BAND_5, 'GAIN_CHANGE_BAND_6_VCID_1':GAIN_CHANGE_BAND_6_VCID_1, 'GAIN_CHANGE_BAND_6_VCID_2':GAIN_CHANGE_BAND_6_VCID_2, 'GAIN_CHANGE_BAND_7':GAIN_CHANGE_BAND_7, 'GAIN_CHANGE_BAND_8':GAIN_CHANGE_BAND_8,'GAIN_CHANGE_SCAN_BAND_1':GAIN_CHANGE_SCAN_BAND_1,'GAIN_CHANGE_SCAN_BAND_2':GAIN_CHANGE_SCAN_BAND_2,'GAIN_CHANGE_SCAN_BAND_3':GAIN_CHANGE_SCAN_BAND_3,'GAIN_CHANGE_SCAN_BAND_4':GAIN_CHANGE_SCAN_BAND_4, 'GAIN_CHANGE_SCAN_BAND_5':GAIN_CHANGE_SCAN_BAND_5,'GAIN_CHANGE_SCAN_BAND_6_VCID_1':GAIN_CHANGE_SCAN_BAND_6_VCID_1,'GAIN_CHANGE_SCAN_BAND_6_VCID_2':GAIN_CHANGE_SCAN_BAND_6_VCID_2, 'GAIN_CHANGE_SCAN_BAND_7':GAIN_CHANGE_SCAN_BAND_7, 'GAIN_CHANGE_SCAN_BAND_8':GAIN_CHANGE_SCAN_BAND_8,'GEOMETRIC_RMSE_MODEL':GEOMETRIC_RMSE_MODEL,'GEOMETRIC_RMSE_MODEL_X':GEOMETRIC_RMSE_MODEL_X,'GEOMETRIC_RMSE_MODEL_Y':GEOMETRIC_RMSE_MODEL_Y,'GRID_CELL_SIZE_PANCHROMATIC':GRID_CELL_SIZE_PANCHROMATIC,'GRID_CELL_SIZE_REFLECTIVE':GRID_CELL_SIZE_REFLECTIVE,'GRID_CELL_SIZE_THERMAL':GRID_CELL_SIZE_THERMAL,'GROUND_CONTROL_POINTS_MODEL':GROUND_CONTROL_POINTS_MODEL, 'GROUND_CONTROL_POINTS_VERSION':GROUND_CONTROL_POINTS_VERSION,'IMAGE_QUALITY':IMAGE_QUALITY,'K1_CONSTANT_BAND_6_VCID_1':K1_CONSTANT_BAND_6_VCID_1,'K1_CONSTANT_BAND_6_VCID_2':K1_CONSTANT_BAND_6_VCID_2,'K2_CONSTANT_BAND_6_VCID_1':K2_CONSTANT_BAND_6_VCID_1, 'K2_CONSTANT_BAND_6_VCID_2':K2_CONSTANT_BAND_6_VCID_2,'LANDSAT_SCENE_ID':LANDSAT_SCENE_ID, 'MAP_PROJECTION':MAP_PROJECTION, 'ORIENTATION':ORIENTATION,'PANCHROMATIC_LINES':PANCHROMATIC_LINES,'PANCHROMATIC_SAMPLES':PANCHROMATIC_SAMPLES,'PROCESSING_SOFTWARE_VERSION':PROCESSING_SOFTWARE_VERSION,'RADIANCE_ADD_BAND_1':RADIANCE_ADD_BAND_1,'RADIANCE_ADD_BAND_2':RADIANCE_ADD_BAND_2,'RADIANCE_ADD_BAND_3':RADIANCE_ADD_BAND_3,'RADIANCE_ADD_BAND_4':RADIANCE_ADD_BAND_4,'RADIANCE_ADD_BAND_5':RADIANCE_ADD_BAND_5,'RADIANCE_ADD_BAND_6_VCID_1':RADIANCE_ADD_BAND_6_VCID_1,'RADIANCE_ADD_BAND_6_VCID_2':RADIANCE_ADD_BAND_6_VCID_2,'RADIANCE_ADD_BAND_7':RADIANCE_ADD_BAND_7,'RADIANCE_ADD_BAND_8':RADIANCE_ADD_BAND_8, 'RADIANCE_MULT_BAND_1':RADIANCE_MULT_BAND_1, 'RADIANCE_MULT_BAND_2':RADIANCE_MULT_BAND_2, 'RADIANCE_MULT_BAND_3':RADIANCE_MULT_BAND_3,'RADIANCE_MULT_BAND_4':RADIANCE_MULT_BAND_4,'RADIANCE_MULT_BAND_5':RADIANCE_MULT_BAND_5,'RADIANCE_MULT_BAND_6_VCID_1':RADIANCE_MULT_BAND_6_VCID_1,'RADIANCE_MULT_BAND_6_VCID_2':RADIANCE_MULT_BAND_6_VCID_2,'RADIANCE_MULT_BAND_7':RADIANCE_MULT_BAND_7,'RADIANCE_MULT_BAND_8':RADIANCE_MULT_BAND_8,'REFLECTANCE_ADD_BAND_1':REFLECTANCE_ADD_BAND_1,'REFLECTANCE_ADD_BAND_2':REFLECTANCE_ADD_BAND_2,'REFLECTANCE_ADD_BAND_3':REFLECTANCE_ADD_BAND_3,'REFLECTANCE_ADD_BAND_4':REFLECTANCE_ADD_BAND_4,'REFLECTANCE_ADD_BAND_5':REFLECTANCE_ADD_BAND_5,'REFLECTANCE_ADD_BAND_7':REFLECTANCE_ADD_BAND_7,'REFLECTANCE_ADD_BAND_8':REFLECTANCE_ADD_BAND_8,'REFLECTANCE_MULT_BAND_1':REFLECTANCE_MULT_BAND_1,'REFLECTANCE_MULT_BAND_2':REFLECTANCE_MULT_BAND_2,'REFLECTANCE_MULT_BAND_3':REFLECTANCE_MULT_BAND_3,'REFLECTANCE_MULT_BAND_4':REFLECTANCE_MULT_BAND_4,'REFLECTANCE_MULT_BAND_5':REFLECTANCE_MULT_BAND_5,'REFLECTANCE_MULT_BAND_7':REFLECTANCE_MULT_BAND_7,'REFLECTANCE_MULT_BAND_8':REFLECTANCE_MULT_BAND_8,'REFLECTIVE_LINES':REFLECTIVE_LINES,'REFLECTIVE_SAMPLES':REFLECTIVE_SAMPLES,'REQUEST_ID':REQUEST_ID,'SATURATION_BAND_1':SATURATION_BAND_1,'SATURATION_BAND_2':SATURATION_BAND_2, 'SATURATION_BAND_3':SATURATION_BAND_3,'SATURATION_BAND_4':SATURATION_BAND_4,'SATURATION_BAND_5':SATURATION_BAND_5,'SATURATION_BAND_6_VCID_1':SATURATION_BAND_6_VCID_1,'SATURATION_BAND_6_VCID_2':SATURATION_BAND_6_VCID_2,'SATURATION_BAND_7':SATURATION_BAND_7,'SATURATION_BAND_8':SATURATION_BAND_8,'SCENE_CENTER_TIME':SCENE_CENTER_TIME,'SENSOR_ID':SENSOR_ID,'SENSOR_MODE':SENSOR_MODE,'SPACECRAFT_ID':SPACECRAFT_ID,'STATION_ID':STATION_ID,'SUN_AZIMUTH':SUN_AZIMUTH,'SUN_ELEVATION':SUN_ELEVATION,'THERMAL_LINES':THERMAL_LINES,'THERMAL_SAMPLES':THERMAL_SAMPLES,'UTM_ZONE':UTM_ZONE,'WRS_PATH':WRS_PATH,'WRS_ROW':WRS_ROW,'index':index, 'time_end':time_end, 'time_start':time_start}
METADATA_DF = pd.DataFrame(data=METADATA_ARRAY)
METADATA_DF.to_csv(path_or_buf= 'C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/landsat7_meta.csv', index= False)
METADATA_DF

,id,CLOUD_COVER,CLOUD_COVER_LAND,COLLECTION_CATEGORY,COLLECTION_NUMBER,CORRECTION_BIAS_BAND_1,CORRECTION_BIAS_BAND_2,CORRECTION_BIAS_BAND_3,CORRECTION_BIAS_BAND_4,CORRECTION_BIAS_BAND_5,...,SUN_AZIMUTH,SUN_ELEVATION,THERMAL_LINES,THERMAL_SAMPLES,UTM_ZONE,WRS_PATH,WRS_ROW,index,time_end,time_start
0,LE07_L1TP_231066_20190813_20190908_01_T1_BOA_GEOM,1,1,T1,1,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,...,54.393925,50.169384,6961,8111,20,231,66,LE07_231066_20190813,1565704908880,1565704908876
1,LE07_L1TP_231066_20190829_20190924_01_T1_BOA_GEOM,0,0,T1,1,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,...,61.075321,53.618484,6961,8111,20,231,66,LE07_231066_20190829,1567087268230,1567087268226
2,LE07_L1TP_231066_20190914_20191028_01_T1_BOA_GEOM,4,4,T1,1,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,...,69.677711,57.221615,6961,8101,20,231,66,LE07_231066_20190914,1568469626210,1568469626208
3,LE07_L1TP_231066_20190930_20191026_01_T1_BOA_GEOM,29,29,T1,1,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,INTERNAL_CALIBRATION,...,80.415825,60.237232,6991,8121,20,231,66,LE07_231066_20190930,1569851984690,1569851984694


In [99]:
## Extract Landsat OLI metadata
LANDSAT_PRODUCT_ID = landsat_collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()
BPF_NAME_OLI = landsat_collection.aggregate_array('BPF_NAME_OLI').getInfo()
BPF_NAME_TIRS = landsat_collection.aggregate_array('BPF_NAME_TIRS').getInfo()
CLOUD_COVER = landsat_collection.aggregate_array('CLOUD_COVER').getInfo()
CLOUD_COVER_LAND = landsat_collection.aggregate_array('CLOUD_COVER_LAND').getInfo()
COLLECTION_CATEGORY = landsat_collection.aggregate_array('COLLECTION_CATEGORY').getInfo()
COLLECTION_NUMBER = landsat_collection.aggregate_array('COLLECTION_NUMBER').getInfo()
CPF_NAME = landsat_collection.aggregate_array('CPF_NAME').getInfo()
DATA_TYPE = landsat_collection.aggregate_array('DATA_TYPE').getInfo()
DATE_ACQUIRED = landsat_collection.aggregate_array('DATE_ACQUIRED').getInfo()
DATUM = landsat_collection.aggregate_array('DATUM').getInfo()
EARTH_SUN_DISTANCE = landsat_collection.aggregate_array('EARTH_SUN_DISTANCE').getInfo()
ELEVATION_SOURCE = landsat_collection.aggregate_array('ELEVATION_SOURCE').getInfo()
ELLIPSOID = landsat_collection.aggregate_array('ELLIPSOID').getInfo()
FILE_DATE = landsat_collection.aggregate_array('FILE_DATE').getInfo()
GEOMETRIC_RMSE_MODEL = landsat_collection.aggregate_array('GEOMETRIC_RMSE_MODEL').getInfo()
GEOMETRIC_RMSE_MODEL_X = landsat_collection.aggregate_array('GEOMETRIC_RMSE_MODEL_X').getInfo()
GEOMETRIC_RMSE_MODEL_Y = landsat_collection.aggregate_array('GEOMETRIC_RMSE_MODEL_Y').getInfo()
GEOMETRIC_RMSE_VERIFY = landsat_collection.aggregate_array('GEOMETRIC_RMSE_VERIFY').getInfo()
GRID_CELL_SIZE_PANCHROMATIC = landsat_collection.aggregate_array('GRID_CELL_SIZE_PANCHROMATIC').getInfo()
GRID_CELL_SIZE_REFLECTIVE = landsat_collection.aggregate_array('GRID_CELL_SIZE_REFLECTIVE').getInfo()
GRID_CELL_SIZE_THERMAL = landsat_collection.aggregate_array('GRID_CELL_SIZE_THERMAL').getInfo()
GROUND_CONTROL_POINTS_MODEL = landsat_collection.aggregate_array('GROUND_CONTROL_POINTS_MODEL').getInfo()
GROUND_CONTROL_POINTS_VERIFY = landsat_collection.aggregate_array('GROUND_CONTROL_POINTS_VERIFY').getInfo()
GROUND_CONTROL_POINTS_VERSION = landsat_collection.aggregate_array('GROUND_CONTROL_POINTS_VERSION').getInfo()
IMAGE_QUALITY_OLI = landsat_collection.aggregate_array('IMAGE_QUALITY_OLI').getInfo()
IMAGE_QUALITY_TIRS = landsat_collection.aggregate_array('IMAGE_QUALITY_TIRS').getInfo()
K1_CONSTANT_BAND_10 = landsat_collection.aggregate_array('K1_CONSTANT_BAND_10').getInfo()
K1_CONSTANT_BAND_11 = landsat_collection.aggregate_array('K1_CONSTANT_BAND_11').getInfo()
K2_CONSTANT_BAND_10 = landsat_collection.aggregate_array('K2_CONSTANT_BAND_10').getInfo()
K2_CONSTANT_BAND_11 = landsat_collection.aggregate_array('K2_CONSTANT_BAND_11').getInfo()
LANDSAT_SCENE_ID = landsat_collection.aggregate_array('LANDSAT_SCENE_ID').getInfo()
MAP_PROJECTION = landsat_collection.aggregate_array('MAP_PROJECTION').getInfo()
NADIR_OFFNADIR = landsat_collection.aggregate_array('NADIR_OFFNADIR').getInfo()
ORIENTATION = landsat_collection.aggregate_array('ORIENTATION').getInfo()
PANCHROMATIC_LINES = landsat_collection.aggregate_array('PANCHROMATIC_LINES').getInfo()
PANCHROMATIC_SAMPLES = landsat_collection.aggregate_array('PANCHROMATIC_SAMPLES').getInfo()
PROCESSING_SOFTWARE_VERSION = landsat_collection.aggregate_array('PROCESSING_SOFTWARE_VERSION').getInfo()
RADIANCE_ADD_BAND_1 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_1').getInfo()
RADIANCE_ADD_BAND_10 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_10').getInfo()
RADIANCE_ADD_BAND_11 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_11').getInfo()
RADIANCE_ADD_BAND_2 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_2').getInfo()
RADIANCE_ADD_BAND_3 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_3').getInfo()
RADIANCE_ADD_BAND_4 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_4').getInfo()
RADIANCE_ADD_BAND_5 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_5').getInfo()
RADIANCE_ADD_BAND_6 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_6').getInfo()
RADIANCE_ADD_BAND_7 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_7').getInfo()
RADIANCE_ADD_BAND_8 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_8').getInfo()
RADIANCE_ADD_BAND_9 = landsat_collection.aggregate_array('RADIANCE_ADD_BAND_9').getInfo()
RADIANCE_MULT_BAND_1 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_1').getInfo()
RADIANCE_MULT_BAND_10 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_10').getInfo()
RADIANCE_MULT_BAND_11 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_11').getInfo()
RADIANCE_MULT_BAND_2 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_2').getInfo()
RADIANCE_MULT_BAND_3 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_3').getInfo()
RADIANCE_MULT_BAND_4 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_4').getInfo()
RADIANCE_MULT_BAND_5 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_5').getInfo()
RADIANCE_MULT_BAND_6 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_6').getInfo()
RADIANCE_MULT_BAND_7 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_7').getInfo()
RADIANCE_MULT_BAND_8 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_8').getInfo()
RADIANCE_MULT_BAND_9 = landsat_collection.aggregate_array('RADIANCE_MULT_BAND_9').getInfo()
REFLECTANCE_ADD_BAND_1 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_1').getInfo()
REFLECTANCE_ADD_BAND_2 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_2').getInfo()
REFLECTANCE_ADD_BAND_3 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_3').getInfo()
REFLECTANCE_ADD_BAND_4 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_4').getInfo()
REFLECTANCE_ADD_BAND_5 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_5').getInfo()
REFLECTANCE_ADD_BAND_6 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_6').getInfo()
REFLECTANCE_ADD_BAND_7 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_7').getInfo()
REFLECTANCE_ADD_BAND_8 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_8').getInfo()
REFLECTANCE_ADD_BAND_9 = landsat_collection.aggregate_array('REFLECTANCE_ADD_BAND_9').getInfo()
REFLECTANCE_MULT_BAND_1 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_1').getInfo()
REFLECTANCE_MULT_BAND_2 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_2').getInfo()
REFLECTANCE_MULT_BAND_3 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_3').getInfo()
REFLECTANCE_MULT_BAND_4 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_4').getInfo()
REFLECTANCE_MULT_BAND_5 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_5').getInfo()
REFLECTANCE_MULT_BAND_6 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_6').getInfo()
REFLECTANCE_MULT_BAND_7 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_7').getInfo()
REFLECTANCE_MULT_BAND_8 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_8').getInfo()
REFLECTANCE_MULT_BAND_9 = landsat_collection.aggregate_array('REFLECTANCE_MULT_BAND_9').getInfo()
REFLECTIVE_LINES = landsat_collection.aggregate_array('REFLECTIVE_LINES').getInfo()
REFLECTIVE_SAMPLES = landsat_collection.aggregate_array('REFLECTIVE_SAMPLES').getInfo()
REQUEST_ID = landsat_collection.aggregate_array('REQUEST_ID').getInfo()
RLUT_FILE_NAME = landsat_collection.aggregate_array('RLUT_FILE_NAME').getInfo()
ROLL_ANGLE = landsat_collection.aggregate_array('ROLL_ANGLE').getInfo()
SATURATION_BAND_1 = landsat_collection.aggregate_array('SATURATION_BAND_1').getInfo()
SATURATION_BAND_2 = landsat_collection.aggregate_array('SATURATION_BAND_2').getInfo()
SATURATION_BAND_3 = landsat_collection.aggregate_array('SATURATION_BAND_3').getInfo()
SATURATION_BAND_4 = landsat_collection.aggregate_array('SATURATION_BAND_4').getInfo()
SATURATION_BAND_5 = landsat_collection.aggregate_array('SATURATION_BAND_5').getInfo()
SATURATION_BAND_6 = landsat_collection.aggregate_array('SATURATION_BAND_6').getInfo()
SATURATION_BAND_7 = landsat_collection.aggregate_array('SATURATION_BAND_7').getInfo()
SATURATION_BAND_8 = landsat_collection.aggregate_array('SATURATION_BAND_8').getInfo()
SATURATION_BAND_9 = landsat_collection.aggregate_array('SATURATION_BAND_9').getInfo()
SCENE_CENTER_TIME = landsat_collection.aggregate_array('SCENE_CENTER_TIME').getInfo()
SENSOR_ID = landsat_collection.aggregate_array('SENSOR_ID').getInfo()
SPACECRAFT_ID = landsat_collection.aggregate_array('SPACECRAFT_ID').getInfo()
STATION_ID = landsat_collection.aggregate_array('STATION_ID').getInfo()
SUN_AZIMUTH = landsat_collection.aggregate_array('SUN_AZIMUTH').getInfo()
SUN_ELEVATION = landsat_collection.aggregate_array('SUN_ELEVATION').getInfo()
TARGET_WRS_PATH = landsat_collection.aggregate_array('TARGET_WRS_PATH').getInfo()
TARGET_WRS_ROW = landsat_collection.aggregate_array('TARGET_WRS_ROW').getInfo()
THERMAL_LINES = landsat_collection.aggregate_array('THERMAL_LINES').getInfo()
THERMAL_SAMPLES = landsat_collection.aggregate_array('THERMAL_SAMPLES').getInfo()
TIRS_SSM_MODEL = landsat_collection.aggregate_array('TIRS_SSM_MODEL').getInfo()
TIRS_SSM_POSITION_STATUS = landsat_collection.aggregate_array('TIRS_SSM_POSITION_STATUS').getInfo()
TIRS_STRAY_LIGHT_CORRECTION_SOURCE = landsat_collection.aggregate_array('TIRS_STRAY_LIGHT_CORRECTION_SOURCE').getInfo()
TRUNCATION_OLI = landsat_collection.aggregate_array('TRUNCATION_OLI').getInfo()
UTM_ZONE = landsat_collection.aggregate_array('UTM_ZONE').getInfo()
WRS_PATH = landsat_collection.aggregate_array('WRS_PATH').getInfo()
WRS_ROW = landsat_collection.aggregate_array('WRS_ROW').getInfo()
index = landsat_collection.aggregate_array('system:index').getInfo()
time_end = landsat_collection.aggregate_array('system:time_end').getInfo()
time_start = landsat_collection.aggregate_array('system:time_start').getInfo()

## Insert suffix 
LANDSAT_PRODUCT_ID = [s + '_BOA_GEOM' for s in LANDSAT_PRODUCT_ID]

## Export GEE Sentinel metadata as CSV 
METADATA_ARRAY = {'id': LANDSAT_PRODUCT_ID, 'BPF_NAME_OLI': BPF_NAME_OLI, 'BPF_NAME_TIRS' : BPF_NAME_TIRS, 'CLOUD_COVER' : CLOUD_COVER, 'CLOUD_COVER_LAND' : CLOUD_COVER_LAND, 'COLLECTION_CATEGORY' : COLLECTION_CATEGORY, 'COLLECTION_NUMBER' : COLLECTION_NUMBER, 'CPF_NAME' :CPF_NAME, 'DATA_TYPE' : DATA_TYPE, 'DATE_ACQUIRED' : DATE_ACQUIRED, 'DATUM' : DATUM, 'EARTH_SUN_DISTANCE' : EARTH_SUN_DISTANCE, 'ELEVATION_SOURCE' : ELEVATION_SOURCE, 'ELLIPSOID' : ELLIPSOID, 'FILE_DATE' : FILE_DATE, 'GEOMETRIC_RMSE_MODEL' : GEOMETRIC_RMSE_MODEL , 'GEOMETRIC_RMSE_MODEL_X' : GEOMETRIC_RMSE_MODEL_X, 'GEOMETRIC_RMSE_MODEL_Y' : GEOMETRIC_RMSE_MODEL_Y, 'GEOMETRIC_RMSE_VERIFY' :GEOMETRIC_RMSE_VERIFY, 'GRID_CELL_SIZE_PANCHROMATIC' :GRID_CELL_SIZE_PANCHROMATIC, 'GRID_CELL_SIZE_REFLECTIVE' :GRID_CELL_SIZE_REFLECTIVE, 'GRID_CELL_SIZE_THERMAL' : GRID_CELL_SIZE_THERMAL, 'GROUND_CONTROL_POINTS_MODEL' : GROUND_CONTROL_POINTS_MODEL, 'GROUND_CONTROL_POINTS_VERIFY' : GROUND_CONTROL_POINTS_VERIFY, 'GROUND_CONTROL_POINTS_VERSION' : GROUND_CONTROL_POINTS_VERSION, 'IMAGE_QUALITY_OLI' : IMAGE_QUALITY_OLI, 'IMAGE_QUALITY_TIRS' : IMAGE_QUALITY_TIRS, 'K1_CONSTANT_BAND_10' : K1_CONSTANT_BAND_10, 'K1_CONSTANT_BAND_11' : K1_CONSTANT_BAND_11, 'K2_CONSTANT_BAND_10' : K2_CONSTANT_BAND_10, 'K2_CONSTANT_BAND_11' : K2_CONSTANT_BAND_11, 'MAP_PROJECTION' : MAP_PROJECTION,  'NADIR_OFFNADIR' : NADIR_OFFNADIR, 'ORIENTATION' : ORIENTATION,  'PANCHROMATIC_LINES' : PANCHROMATIC_LINES, 'PANCHROMATIC_SAMPLES' : PANCHROMATIC_SAMPLES, 'PROCESSING_SOFTWARE_VERSION' : PROCESSING_SOFTWARE_VERSION, 'RADIANCE_ADD_BAND_1' : RADIANCE_ADD_BAND_1, 'RADIANCE_ADD_BAND_10' : RADIANCE_ADD_BAND_10, 'RADIANCE_ADD_BAND_11' : RADIANCE_ADD_BAND_11, 'RADIANCE_ADD_BAND_2' : RADIANCE_ADD_BAND_2, 'RADIANCE_ADD_BAND_3' : RADIANCE_ADD_BAND_3, 'RADIANCE_ADD_BAND_4' : RADIANCE_ADD_BAND_4, 'RADIANCE_ADD_BAND_5' : RADIANCE_ADD_BAND_5, 'RADIANCE_ADD_BAND_6' : RADIANCE_ADD_BAND_6, 'RADIANCE_ADD_BAND_7' : RADIANCE_ADD_BAND_7, 'RADIANCE_ADD_BAND_8' : RADIANCE_ADD_BAND_8, 'RADIANCE_ADD_BAND_9' :RADIANCE_ADD_BAND_9 , 'RADIANCE_MULT_BAND_1': RADIANCE_MULT_BAND_1, 'RADIANCE_MULT_BAND_10' : RADIANCE_MULT_BAND_10 ,'RADIANCE_MULT_BAND_11': RADIANCE_MULT_BAND_11, 'RADIANCE_MULT_BAND_2': RADIANCE_MULT_BAND_2, 'RADIANCE_MULT_BAND_3' : RADIANCE_MULT_BAND_3, 'RADIANCE_MULT_BAND_4' : RADIANCE_MULT_BAND_4, 'RADIANCE_MULT_BAND_5' : RADIANCE_MULT_BAND_5, 'RADIANCE_MULT_BAND_6' : RADIANCE_MULT_BAND_6, 'RADIANCE_MULT_BAND_7' : RADIANCE_MULT_BAND_7, 'RADIANCE_MULT_BAND_8' : RADIANCE_MULT_BAND_8, 'RADIANCE_MULT_BAND_9': RADIANCE_MULT_BAND_9, 'REFLECTANCE_ADD_BAND_1' : REFLECTANCE_ADD_BAND_1, 'REFLECTANCE_ADD_BAND_2' : REFLECTANCE_ADD_BAND_2, 'REFLECTANCE_ADD_BAND_3': REFLECTANCE_ADD_BAND_3, 'REFLECTANCE_ADD_BAND_4' : REFLECTANCE_ADD_BAND_4, 'REFLECTANCE_ADD_BAND_5' : REFLECTANCE_ADD_BAND_5, 'REFLECTANCE_ADD_BAND_6' : REFLECTANCE_ADD_BAND_6, 'REFLECTANCE_ADD_BAND_7': REFLECTANCE_ADD_BAND_7, 'REFLECTANCE_ADD_BAND_8' : REFLECTANCE_ADD_BAND_8, 'REFLECTANCE_ADD_BAND_9':REFLECTANCE_ADD_BAND_9 , 'REFLECTANCE_MULT_BAND_1' : REFLECTANCE_MULT_BAND_1, 'REFLECTANCE_MULT_BAND_2' : REFLECTANCE_MULT_BAND_2, 'REFLECTANCE_MULT_BAND_3' : REFLECTANCE_MULT_BAND_3, 'REFLECTANCE_MULT_BAND_4': REFLECTANCE_MULT_BAND_4, 'REFLECTANCE_MULT_BAND_5' : REFLECTANCE_MULT_BAND_5, 'REFLECTANCE_MULT_BAND_6': REFLECTANCE_MULT_BAND_6, 'REFLECTANCE_MULT_BAND_7':REFLECTANCE_MULT_BAND_7, 'REFLECTANCE_MULT_BAND_8':REFLECTANCE_MULT_BAND_8, 'REFLECTANCE_MULT_BAND_9':REFLECTANCE_MULT_BAND_9, 'REFLECTIVE_LINES' : REFLECTIVE_LINES, 'REFLECTIVE_SAMPLES':REFLECTIVE_SAMPLES,'REQUEST_ID': REQUEST_ID, 'RLUT_FILE_NAME':RLUT_FILE_NAME, 'ROLL_ANGLE':ROLL_ANGLE,'SATURATION_BAND_1' : SATURATION_BAND_1, 'SATURATION_BAND_2':SATURATION_BAND_2, 'SATURATION_BAND_3':SATURATION_BAND_3, 'SATURATION_BAND_4':SATURATION_BAND_4, 'SATURATION_BAND_5':SATURATION_BAND_5, 'SATURATION_BAND_6':SATURATION_BAND_6, 'SATURATION_BAND_7':SATURATION_BAND_7, 'SATURATION_BAND_8':SATURATION_BAND_8, 'SATURATION_BAND_9':SATURATION_BAND_9, 'SCENE_CENTER_TIME':SCENE_CENTER_TIME, 'SENSOR_ID':SENSOR_ID, 'SPACECRAFT_ID':SPACECRAFT_ID, 'STATION_ID':STATION_ID, 'SUN_AZIMUTH':SUN_AZIMUTH,  'SUN_ELEVATION':SUN_ELEVATION, 'TARGET_WRS_PATH':TARGET_WRS_PATH, 'TARGET_WRS_ROW':TARGET_WRS_ROW, 'THERMAL_LINES':THERMAL_LINES,'THERMAL_SAMPLES':THERMAL_SAMPLES, 'TIRS_SSM_MODEL':TIRS_SSM_MODEL, 'TIRS_SSM_POSITION_STATUS':TIRS_SSM_POSITION_STATUS, 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE':TIRS_STRAY_LIGHT_CORRECTION_SOURCE, 'TRUNCATION_OLI':TRUNCATION_OLI,'UTM_ZONE':UTM_ZONE,'WRS_PATH':WRS_PATH,  'WRS_ROW':WRS_ROW,'index':index,'time_end':time_end,'time_start':time_start }  
METADATA_DF = pd.DataFrame(data=METADATA_ARRAY)
METADATA_DF.to_csv(path_or_buf= 'C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/landsat8_meta.csv', index= False)
METADATA_DF


,id,BPF_NAME_OLI,BPF_NAME_TIRS,CLOUD_COVER,CLOUD_COVER_LAND,COLLECTION_CATEGORY,COLLECTION_NUMBER,CPF_NAME,DATA_TYPE,DATE_ACQUIRED,...,TIRS_SSM_MODEL,TIRS_SSM_POSITION_STATUS,TIRS_STRAY_LIGHT_CORRECTION_SOURCE,TRUNCATION_OLI,UTM_ZONE,WRS_PATH,WRS_ROW,index,time_end,time_start
0,LC08_L1TP_231066_20190805_20190820_01_T1_BOA_GEOM,LO8BPF20190805134231_20190805152100.01,LT8BPF20190730110028_20190816042437.01,0.000000,0.000000,T1,1,LC08CPF_20190701_20190930_01.02,L1TP,2019-08-05,...,FINAL,ESTIMATED,TIRS,UPPER,20,231,66,LC08_231066_20190805,1565014476840,1565014476844
1,LC08_L1TP_231066_20190821_20190903_01_T1_BOA_GEOM,LO8BPF20190821134324_20190821152217.01,LT8BPF20190816064533_20190830113009.01,44.200001,44.200001,T1,1,LC08CPF_20190701_20190930_01.02,L1TP,2019-08-21,...,FINAL,ESTIMATED,TIRS,UPPER,20,231,66,LC08_231066_20190821,1566396881750,1566396881754
2,LC08_L1TP_231066_20190906_20190917_01_T1_BOA_GEOM,LO8BPF20190906134526_20190906152419.01,LT8BPF20190830113825_20190914152935.01,30.559999,30.559999,T1,1,LC08CPF_20190701_20190930_01.02,L1TP,2019-09-06,...,FINAL,ESTIMATED,TIRS,UPPER,20,231,66,LC08_231066_20190906,1567779284980,1567779284983


In [7]:
## Move files to sensor folders
## Define scene path
ro.r('path <- "C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/2_arosics_GEOM/"')

## Move Sentinel files
ro.r('file.move(list.files(path, pattern = "L1C_", full.names = TRUE), paste0(path, "/sentinel"))')

## Move Landsat 7 (ETM+) files
ro.r('file.move(list.files(path, pattern = "LE07_", full.names = TRUE), paste0(path, "/landsat7"))')

## Move Landsat 8 (OLI) files
ro.r('file.move(list.files(path, pattern = "LC08_", full.names = TRUE), paste0(path, "/landsat8"))')

1,1,1


In [100]:
## Ingest local Sentinel raster files as GEE imageCollection
## Before execute, create specific folders for each sensor  ##!! I M P O R T A N T !!##

## --source [Full name path with al raster files]
## -dest [Asset destination in GEE]
## --metadata [CSV file (previous created) with metadata to be assigned]
## --user [E-mail to acess GEE]
## --nodata [noDATA value, default -9999]

! geebam upload --source C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/2_arosics_GEOM/sentinel --dest users/shps/PNCA_sentinel --metadata C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/sentinel_meta.csv --user dhemerson.conciani@unesp.br --nodata -9999

2020-07-22 10:14:05,060 :: googleapiclient.discovery_cache :: WARNING :: file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <modu

In [101]:
## Ingest local Landsat ETM+ raster files as GEE imageCollection
! geebam upload --source C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/2_arosics_GEOM/landsat7 --dest users/shps/PNCA_landsat7 --metadata C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/landsat7_meta.csv --user dhemerson.conciani@unesp.br --nodata -9999

2020-07-22 10:16:55,776 :: googleapiclient.discovery_cache :: WARNING :: file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <modu

In [102]:
## Ingest local Landsat OLI raster files as GEE imageCollection
! geebam upload --source C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/2_arosics_GEOM/landsat8 --dest users/shps/PNCA_landsat8 --metadata C:/Users/BSD/Desktop/arosics/gee-atmcorr-S2-master/images/landsat8_meta.csv --user dhemerson.conciani@unesp.br --nodata -9999

2020-07-22 10:19:30,432 :: googleapiclient.discovery_cache :: WARNING :: file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BSD\anaconda3\envs\arosics\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <modu

In [103]:
print ('Available images in the period:')
# Sentinel 2A/2B collection images
S2_col = ee.ImageCollection('users/shps/PNCA_sentinel')
col_length_S2 = S2_col.size().getInfo()
print ('S2A/2B images = ', col_length_S2)

# Landsat ETM+ collection images
L7_col = ee.ImageCollection('users/shps/PNCA_landsat7')
col_length_L7= L7_col.size().getInfo()
print ('L7 images = ', col_length_L7)

# Landsat OLI collection images
L8_col = ee.ImageCollection('users/shps/PNCA_landsat8')
col_length_L8 = L8_col.size().getInfo()
print ('L8 images = ', col_length_L8)

Available images in the period:
S2A/2B images =  11
L7 images =  4
L8 images =  3


In [108]:
## BAND ASSIGNATIONS AND BRDF BASIC PARAMETERS

# Input and output band assgnations definition
bandIn_L8  = ['b1','b2','b3','b4','b5','b6']
bandOut_L8 = ['blue','green','red','nir','swir1','swir2']

bandIn_L7  = ['b1','b2','b3','b4','b5','b6']
bandOut_L7 = ['blue','green','red','nir','swir1','swir2']

bandIn_S2  = ['b1','b2','b3','b4','b5','b6']
bandOut_S2 = ['blue','green','red','nir','swir1','swir2']

## BRDF parameters
PI = ee.Number(3.14159265359)
MAX_SATELLITE_ZENITH = 7.5
MAX_DISTANCE = 1000000
UPPER_LEFT = 0
LOWER_LEFT = 1
LOWER_RIGHT = 2
UPPER_RIGHT = 3

In [109]:
## BRDF CORRECTION FUNCIONS DEFINITION

def applyBRDF_Landsat(image):
    info = image.getInfo()['properties'] ##added
    date_fromimage = datetime.datetime.utcfromtimestamp(info['time_start']/1000) ##added
    date = ee.Date(date_fromimage.strftime('%Y-%m-%d')) ##added
    footprint = ee.List(image.geometry().bounds().bounds().coordinates().get(0))
    angles =  getsunAngles(date, footprint)
    sunAz = angles[0]
    sunZen = angles[1]
  
    viewAz = azimuth(footprint)
    viewZen = zenith(footprint)
  
  
    kval = _kvol(sunAz, sunZen, viewAz, viewZen)
    kvol = kval[0]
    kvol0 = kval[1]
    result = _apply_Landsat(image, kvol.multiply(PI), kvol0.multiply(PI))
  
    return result

def applyBRDF_Sentinel(image):
    info = image.getInfo()['properties'] ##added
    date_fromimage = datetime.datetime.utcfromtimestamp(info['time_start']/1000) ##added
    date = ee.Date(date_fromimage.strftime('%Y-%m-%d')) ##added
    footprint = ee.List(image.geometry().bounds().bounds().coordinates().get(0))
    angles =  getsunAngles(date, footprint)
    sunAz = angles[0]
    sunZen = angles[1]
  
    viewAz = azimuth(footprint)
    viewZen = zenith(footprint)
    
    kval = _kvol(sunAz, sunZen, viewAz, viewZen)
    kvol = kval[0]
    kvol0 = kval[1]
    result = _apply_Sentinel(image, kvol.multiply(PI), kvol0.multiply(PI))
  
    return result

def getsunAngles(date, footprint):
    jdp = date.getFraction('year')
    seconds_in_hour = 3600
    hourGMT = ee.Number(date.getRelative('second', 'day')).divide(seconds_in_hour)
    
    latRad = ee.Image.pixelLonLat().select('latitude').multiply(PI.divide(180))
    longDeg = ee.Image.pixelLonLat().select('longitude')
    
    # Julian day proportion in radians
    jdpr = jdp.multiply(PI).multiply(2)
    
    a = ee.List([0.000075, 0.001868, 0.032077, 0.014615, 0.040849])
    meanSolarTime = longDeg.divide(15.0).add(ee.Number(hourGMT))
    localSolarDiff1 = value(a, 0).add(value(a, 1).multiply(jdpr.cos())).subtract(value(a, 2).multiply(jdpr.sin())).subtract(value(a, 3).multiply(jdpr.multiply(2).cos())).subtract(value(a, 4).multiply(jdpr.multiply(2).sin()))
    
    localSolarDiff2 = localSolarDiff1.multiply(12 * 60)
  
    localSolarDiff = localSolarDiff2.divide(PI)
    trueSolarTime = meanSolarTime.add(localSolarDiff.divide(60)).subtract(12.0)
    
    # Hour as an angle
    ah = trueSolarTime.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2).multiply(PI.divide(180)))
    b = ee.List([0.006918, 0.399912, 0.070257, 0.006758, 0.000907, 0.002697, 0.001480])
    delta = value(b, 0).subtract(value(b, 1).multiply(jdpr.cos())).add(value(b, 2).multiply(jdpr.sin())).subtract(value(b, 3).multiply(jdpr.multiply(2).cos())).add(value(b, 4).multiply(jdpr.multiply(2).sin())).subtract(value(b, 5).multiply(jdpr.multiply(3).cos())).add(value(b, 6).multiply(jdpr.multiply(3).sin()))

    cosSunZen = latRad.sin().multiply(delta.sin()).add(latRad.cos().multiply(ah.cos()).multiply(delta.cos()))
    sunZen = cosSunZen.acos()

    # sun azimuth from south, turning west
    sinSunAzSW = ah.sin().multiply(delta.cos()).divide(sunZen.sin())
    sinSunAzSW = sinSunAzSW.clamp(-1.0, 1.0)
  
    cosSunAzSW = (latRad.cos().multiply(-1).multiply(delta.sin()).add(latRad.sin().multiply(delta.cos()).multiply(ah.cos()))).divide(sunZen.sin())
    sunAzSW = sinSunAzSW.asin()
  
    sunAzSW = where(cosSunAzSW.lte(0), sunAzSW.multiply(-1).add(PI), sunAzSW)
    sunAzSW = where(cosSunAzSW.gt(0).And(sinSunAzSW.lte(0)), sunAzSW.add(PI.multiply(2)), sunAzSW) # changed .and to .And
  
    sunAz = sunAzSW.add(PI)
    # Keep within [0, 2pi] range
    sunAz = where(sunAz.gt(PI.multiply(2)), sunAz.subtract(PI.multiply(2)), sunAz)
  
    footprint_polygon = ee.Geometry.Polygon(footprint)
    sunAz = sunAz.clip(footprint_polygon)
    sunAz = sunAz.rename(['sunAz'])
    sunZen = sunZen.clip(footprint_polygon).rename(['sunZen'])
  
    return [sunAz, sunZen]

def azimuth(footprint):
    def x (point):return ee.Number(ee.List(point).get(0))
    def y (point):return ee.Number(ee.List(point).get(1))
    
    upperCenter = line_from_coords(footprint, UPPER_LEFT, UPPER_RIGHT).centroid().coordinates()
    lowerCenter = line_from_coords(footprint, LOWER_LEFT, LOWER_RIGHT).centroid().coordinates()
    slope = ((y(lowerCenter)).subtract(y(upperCenter))).divide((x(lowerCenter)).subtract(x(upperCenter)))
    slopePerp = ee.Number(-1).divide(slope);
    azimuthLeft = ee.Image(PI.divide(2).subtract((slopePerp).atan()))
    
    return azimuthLeft.rename(['viewAz'])

def zenith(footprint):
    leftLine = line_from_coords(footprint, UPPER_LEFT, LOWER_LEFT)
    rightLine = line_from_coords(footprint, UPPER_RIGHT, LOWER_RIGHT)
    leftDistance = ee.FeatureCollection(leftLine).distance(MAX_DISTANCE)
    rightDistance = ee.FeatureCollection(rightLine).distance(MAX_DISTANCE)
    viewZenith = rightDistance.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2)).divide(rightDistance.add(leftDistance)).subtract(ee.Number(MAX_SATELLITE_ZENITH)).clip(ee.Geometry.Polygon(footprint)).rename(['viewZen'])
    
    return viewZenith.multiply(PI.divide(180))

def _apply_Sentinel(image, kvol, kvol0):
    f_iso = 0
    f_geo = 0
    f_vol = 0
    
    blue = _correct_band(image, 'blue', kvol, kvol0, f_iso=0.0774, f_geo=0.0079, f_vol=0.0372)
    green = _correct_band(image, 'green', kvol, kvol0, f_iso=0.1306, f_geo=0.0178, f_vol=0.0580)
    red = _correct_band(image, 'red', kvol, kvol0, f_iso=0.1690, f_geo=0.0227, f_vol=0.0574)
    #re1 = _correct_band(image, 're1', kvol, kvol0, f_iso=0.2085, f_geo=0.0256, f_vol=0.0845)
    #re2 = _correct_band(image, 're2', kvol, kvol0, f_iso=0.2316, f_geo=0.0273, f_vol=0.1003)
    #re3 = _correct_band(image, 're3', kvol, kvol0, f_iso=0.2599, f_geo=0.0294, f_vol=0.1197)
    nir = _correct_band(image, 'nir', kvol, kvol0, f_iso=0.3093, f_geo=0.0330, f_vol=0.1535)
    #re4 = _correct_band(image, 're4', kvol, kvol0, f_iso=0.2907, f_geo=0.0410, f_vol=0.1611)
    swir1 = _correct_band(image, 'swir1', kvol, kvol0, f_iso=0.3430, f_geo=0.0453, f_vol=0.1154) 
    swir2 = _correct_band(image, 'swir2', kvol, kvol0, f_iso=0.2658, f_geo=0.0387, f_vol=0.0639)
        
    #return image.select([]).addBands([blue, green, red,re1,re2,re3,nir,re4,swir1, swir2])
    return image.select([]).addBands([blue, green, red, nir, swir1, swir2])

def _apply_Landsat(image, kvol, kvol0):
    f_iso = 0
    f_geo = 0
    f_vol = 0
    
    blue = _correct_band(image, 'blue', kvol, kvol0, f_iso=0.0774, f_geo=0.0079, f_vol=0.0372)
    green = _correct_band(image, 'green', kvol, kvol0, f_iso=0.1306, f_geo=0.0178, f_vol=0.0580)
    red = _correct_band(image, 'red', kvol, kvol0, f_iso=0.1690, f_geo=0.0227, f_vol=0.0574)
    nir = _correct_band(image, 'nir', kvol, kvol0, f_iso=0.3093, f_geo=0.0330, f_vol=0.1535)
    swir1 = _correct_band(image, 'swir1', kvol, kvol0, f_iso=0.3430, f_geo=0.0453, f_vol=0.1154)   
    swir2 = _correct_band(image, 'swir2', kvol, kvol0, f_iso=0.2658, f_geo=0.0387, f_vol=0.0639)
        
    return image.select([]).addBands([blue, green, red, nir, swir1, swir2])

def _correct_band(image, band_name, kvol, kvol0, f_iso, f_geo, f_vol):
    ##"""fiso + fvol * kvol + fgeo * kgeo"""
    iso = ee.Image(f_iso)
    geo = ee.Image(f_geo)
    vol = ee.Image(f_vol)
    pred = vol.multiply(kvol).add(geo.multiply(kvol)).add(iso).rename(['pred'])
    pred0 = vol.multiply(kvol0).add(geo.multiply(kvol0)).add(iso).rename(['pred0'])
    cfac = pred0.divide(pred).rename(['cfac'])
    corr = image.select(band_name).multiply(cfac).rename([band_name])
    
    return corr

def _kvol(sunAz, sunZen, viewAz, viewZen):
    ##"""Calculate kvol kernel.
    ##From Lucht et al. 2000
    ##Phase angle = cos(solar zenith) cos(view zenith) + sin(solar zenith) sin(view zenith) cos(relative azimuth)"""

    relative_azimuth = sunAz.subtract(viewAz).rename(['relAz'])
    pa1 = viewZen.cos().multiply(sunZen.cos())
    pa2 = viewZen.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle1 = pa1.add(pa2)
    phase_angle = phase_angle1.acos()
    p1 = ee.Image(PI.divide(2)).subtract(phase_angle)
    p2 = p1.multiply(phase_angle1)
    p3 = p2.add(phase_angle.sin())
    p4 = sunZen.cos().add(viewZen.cos())
    p5 = ee.Image(PI.divide(4))

    kvol = p3.divide(p4).subtract(p5).rename(['kvol'])

    viewZen0 = ee.Image(0)
    pa10 = viewZen0.cos().multiply(sunZen.cos())
    pa20 = viewZen0.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle10 = pa10.add(pa20)
    phase_angle0 = phase_angle10.acos()
    p10 = ee.Image(PI.divide(2)).subtract(phase_angle0)
    p20 = p10.multiply(phase_angle10)
    p30 = p20.add(phase_angle0.sin())
    p40 = sunZen.cos().add(viewZen0.cos())
    p50 = ee.Image(PI.divide(4))

    kvol0 = p30.divide(p40).subtract(p50).rename(['kvol0'])

    return [kvol, kvol0]

def line_from_coords(coordinates, fromIndex, toIndex):
    return ee.Geometry.LineString(ee.List([
      coordinates.get(fromIndex),
      coordinates.get(toIndex)]))

def where(condition, trueValue, falseValue):
    trueMasked = trueValue.mask(condition)
    falseMasked = falseValue.mask(invertMask(condition))
    
    return trueMasked.unmask(falseMasked)

def invertMask(mask):
    return mask.multiply(-1).add(1)

def value(list,index):
    return ee.Number(list.get(index))

In [106]:
# APPLY BRDF CORRECTIONS TO LANDSAT 8 COLLECTION
for i in range (0,col_length_L8):
       
    L8_list = L8_col.toList(col_length_L8)
    image = ee.Image(L8_list.get(i)).select(bandIn_L8,bandOut_L8)
    fname = ee.String(image.get('id')).getInfo()
    #fname = str(fname [12:20]) + '_L08'
    
    image_BOA_BRDF = applyBRDF_Landsat(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BRDF',
        'folder':'BRDF_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '_BRDF.tif' + '--->done')

exporting LC08_L1TP_231066_20190805_20190820_01_T1_BOA_GEOM_BRDF.tif--->done
exporting LC08_L1TP_231066_20190821_20190903_01_T1_BOA_GEOM_BRDF.tif--->done
exporting LC08_L1TP_231066_20190906_20190917_01_T1_BOA_GEOM_BRDF.tif--->done


In [110]:
# APPLY BRDF CORRECTIONS TO LANDSAT 7 COLLECTION
for i in range (0,col_length_L7):
       
    L7_list = L7_col.toList(col_length_L7)
    image = ee.Image(L7_list.get(i)).select(bandIn_L7,bandOut_L7)
    fname = ee.String(image.get('id')).getInfo()
    #fname = str(fname [12:20]) + '_L07'
    
    image_BOA_BRDF = applyBRDF_Landsat(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BRDF',
        'folder':'BRDF_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '_BRDF.tif' + '--->done')

exporting LE07_L1TP_231066_20190813_20190908_01_T1_BOA_GEOM_BRDF.tif--->done
exporting LE07_L1TP_231066_20190829_20190924_01_T1_BOA_GEOM_BRDF.tif--->done
exporting LE07_L1TP_231066_20190914_20191028_01_T1_BOA_GEOM_BRDF.tif--->done
exporting LE07_L1TP_231066_20190930_20191026_01_T1_BOA_GEOM_BRDF.tif--->done


In [111]:
# APPLY BRDF CORRECTIONS TO SENTINEL 2 COLLECTION
for i in range (0,col_length_S2):
       
    S2_list = S2_col.toList(col_length_S2)
    image = ee.Image(S2_list.get(i)).select(bandIn_S2,bandOut_S2)
    fname = image.get('id').getInfo()
   # fname = str(fname [25:33]) + '_' + str(fname [0:3])
    
    image_BOA_BRDF = applyBRDF_Sentinel(image)
    task = ee.batch.Export.image.toDrive(**{
        'image': image_BOA_BRDF,
        'description':fname + '_BRDF',
        'folder':'BRDF_export',
        'scale': 20,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '_BRDF.tif' + '--->done')

exporting L1C_T20LPR_A012663_20190809T142759_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A012806_20190819T142758_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A012949_20190829T142756_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A013092_20190908T142754_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A013235_20190918T143240_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A013378_20190928T142753_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A021500_20190804T143101_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A021643_20190814T142756_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A021786_20190824T142754_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A021929_20190903T142959_BOA_GEOM_BRDF.tif--->done
exporting L1C_T20LPR_A022072_20190913T142751_BOA_GEOM_BRDF.tif--->done


In [ ]:
## REFERENCES ##

# -- BRDF functions
# ROY, D. P.; ZHANG, H. K.; JU, J.; GOMEZ-DANS, J. L.; LEWIS, P. E.; SCHAAF, C. B.; SUN, Q.; LI, J.; HUANG, H.; KOVALSKYY, V. A general method 
# to normalize Landsat reflectance data to nadir BRDF adjusted reflectance. Remote Sensing of Environment, v. 176, p. 255–271, 2016. 
# ROY, D. P.; LI, J.; ZHANG, H. K.; YAN, L.; HUANG, H.; LI, Z. Examination of Sentinel-2A multi-spectral instrument (MSI) reflectance anisotropy 
# and the suitability of a general method to normalize MSI reflectance to nadir BRDF adjusted reflectance. Remote Sensing of Environment, 
# v. 199, p. 25–38, set. 2017.
# ZHANG, H. K.; ROY, D. P.; YAN, L.; LI, Z.; HUANG, H.; VERMOTE, E.; SKAKUN, S.; ROGER, J.-C. Characterization of Sentinel-2A and Landsat-8 top 
# of atmosphere, surface, and nadir BRDF adjusted reflectance and NDVI differences. Remote Sensing of Environment, v. 215, p. 482–494,  2018.

# -- Harmonization of Landsat-Sentinel2 examples / scripts examples used here
# NGUYEN, M. D.; BAEZ-VILLANUEVA, O. M.; BUI, D. D.; NGUYEN, P. T.; RIBBE, L. Harmonization of Landsat and Sentinel 2 for crop monitoring 
# in drought prone areas: Case studies of Ninh Thuan (Vietnam) and Bekaa (Lebanon). Remote Sensing, v. 12, n. 2, p. 1–18, 2020.
# POORTINGA, A.; TENNESON, K.; SHAPIRO, A.; NQUYEN, Q.; SAN AUNG, K.; CHISHTIE, F.; SAAH, D. Mapping Plantations in Myanmar by Fusing 
# Landsat-8, Sentinel-2 and Sentinel-1 Data along with Systematic Error Quantification. Remote Sensing, v. 11, n. 7, p. 831, 2019.